# Get the theoretical price of `$XEG`via intraday data
my friends version `DK_Real time ETF price from holdings.ipynb` seems to be more accurate

In [1]:
# Importing the libraries
import math
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 9)   # Change the size of plots

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# !pip yfinance
import yfinance as yf

# https://github.com/atreadw1492/yahoo_fin
# !pip install yahoo_fin
from yahoo_fin.stock_info import * # get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table 

# !pip install ta
import ta 

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
# get_quote_table('aapl')
# get_data('aapl')

In [3]:
start = datetime.datetime(2016,1,1)
# start = datetime.datetime(2020,3,21)
end = datetime.date.today()

Billion = 1000000000  
Million = 1000000  

def get_marketcap(symbol):
    tmp = get_quote_table(symbol)['Market Cap']
    if 'B' in tmp:
        return  Billion * float(tmp[:-1])
    elif 'M' in tmp:
        return  Million * float(tmp[:-1])
    else:
        print(f'What????, {tmp}')

In [4]:
# Correct as of 18 may https://www.blackrock.com/ca/investors/en/products/239839/ishares-sptsx-capped-energy-index-etf
symbols = ['CNQ.to', 'SU.to', 'CVE.to', 'IMO.to', 'TOU.to', 'ARX.to', 'WCP.to', 'PSK.to', 'CPG.to', 'PXT.to', 'MEG.to', 'ERF.to', 'VET.to']
weights = [25.51, 24.45, 11.96, 8.41, 7.67, 5.90, 2.80, 2.69, 2.56, 2.55, 2.08, 1.89, 1.50] # 0.03 in cash 
weights = np.array(weights)/100 
symbol_weights = dict(zip(symbols, weights))

In [5]:
# dataset = dict()
# dataset['XEG.TO'] = yf.download('XEG.TO', start, end)["Adj Close"]

computed_price = 0.03/100 # cash held     
computed_price_alt = 0.03/100
for symbol in symbols:
#     dataset[symbol] = yf.download(symbol, start, end)["Adj Close"]
   
    marketcap = get_marketcap(symbol)
    price = get_live_price(symbol)
    weight = symbol_weights[symbol]
    
    price_of_this =  0 # or price *weight
    computed_price += (weight * marketcap) / price
    computed_price_alt += weight*price
    print(f'{symbol}\t Weighted price is {price:.2f} * {weight:.2f} = {weight * price:.2f}')
    
# stocks = pd.DataFrame(dataset)
print(f'\nComputed price {computed_price:.2f} Expected price {get_live_price("XEG.to"):.2f}. computed_price_alt {computed_price_alt:.2f}')

CNQ.to	 Weighted price is 45.64 * 0.26 = 11.64
SU.to	 Weighted price is 30.61 * 0.24 = 7.48
CVE.to	 Weighted price is 12.21 * 0.12 = 1.46
IMO.to	 Weighted price is 41.17 * 0.08 = 3.46
TOU.to	 Weighted price is 32.62 * 0.08 = 2.50
ARX.to	 Weighted price is 9.89 * 0.06 = 0.58
WCP.to	 Weighted price is 6.38 * 0.03 = 0.18
PSK.to	 Weighted price is 13.84 * 0.03 = 0.37
CPG.to	 Weighted price is 5.63 * 0.03 = 0.14
PXT.to	 Weighted price is 22.52 * 0.03 = 0.57
MEG.to	 Weighted price is 8.78 * 0.02 = 0.18
ERF.to	 Weighted price is 8.59 * 0.02 = 0.16
VET.to	 Weighted price is 10.85 * 0.01 = 0.16

Computed price 1094474329.62 Expected price 9.15. computed_price_alt 28.91


#### XEG.CA Split History:    Aug 6, 2008 - 4:1

> I didnt know weights were of market cap

In [6]:
# computed_price_adj = computed_price/4
computed_price__alt_adj = computed_price_alt/4

print(f'Computed price {computed_price:.2f} Expected price {get_live_price("XEG.to"):.2f}. computed_price_alt {computed_price__alt_adj:.2f}')

Computed price 1094474329.62 Expected price 9.15. computed_price_alt 7.23


In [7]:
XEG_shares_outstanding = 152500000
computed_price/XEG_shares_outstanding

computed_price__alt_adj

7.176880849948075

7.22803495016098

## Another way...
prices * norm(weights*marketcaps)

In [8]:
myList = [symbol_weights[symbol] * get_marketcap(symbol) for symbol in symbols]
# To correctly normalize
myList.append(358250.70) # * 0.03/100 # cash, 

myList
weights_times_marketcap_normalized = np.array(myList)/sum(myList)
sum(weights_times_marketcap_normalized)
weights_times_marketcap_normalized

[13794787600.0,
 11278296000.0,
 2946106800.0000005,
 2541670200.0000005,
 743069600.0,
 422086000.0,
 112979999.99999999,
 82825100.0,
 83840000.0,
 73491000.0,
 55369600.0,
 41674500.0,
 25935000.0,
 358250.7]

0.9999999999999999

array([4.28376431e-01, 3.50230561e-01, 9.14869264e-02, 7.89277546e-02,
       2.30749115e-02, 1.31072474e-02, 3.50842439e-03, 2.57200921e-03,
       2.60352541e-03, 2.28215274e-03, 1.71941985e-03, 1.29413907e-03,
       8.05372513e-04, 1.11249380e-05])

In [9]:
summer = (0.03/100)
for i in range(len(symbols)): 
    summer += weights_times_marketcap_normalized[i] * get_live_price(symbols[i])
summer/4    

8.920872530455075